# Practice Session 04: Basket analysis

Author: <font color="blue">Jan Aguiló Plana</font>

E-mail: <font color="blue">jan.aguilo01@estudiant.upf.edu</font>

Date: <font color="blue">16/10/2024</font>

In [1]:
import numpy as np  
import matplotlib.pyplot as plt  
import pandas as pd  
import csv
import gzip

from apyori import apriori

# 1. Playing with apyori

In [2]:
# LEAVE AS-IS

def print_apyori_output (association_results, info=False, info_key=False):
    for relation_record in association_results:
        itemset = list(relation_record.items)
        
        # Consider only itemsets of two elements
        if len(itemset) > 1: 
        
            print("Rules involving itemset %s" % itemset)
            support = relation_record.support

            for rules in relation_record.ordered_statistics:
                antecedent = list(rules.items_base)
                consequent = list(rules.items_add)
                
                if info_key:
                    antecedent = [info.loc[x][info_key] for x in antecedent]
                    consequent = [info.loc[x][info_key] for x in consequent]
                
                confidence = rules.confidence
                lift = rules.lift

                print("%s => %s (support=%.4f, confidence=%.2f, lift=%.2f)" %
                      (antecedent, consequent, support, confidence, lift))
            print()

<font size="+1" color="red">Replace this cell with your own example of transactions (at least 20 transactions). Execute the apriori algorithm, in which you should obtain at least <strong>two</strong> rules of the form ['A', 'B'] => ['C'], i.e., at least two rules having a 2-itemset in the antecedent and a 1-itemset in the consequent. Modify the transactions until you obtain such rules.</font>

In [3]:
# I chose transactions that have to do with the market: some of the apps are trading apps that are used to invest money in,
# some of them are news apps specifically for the market, and some others are just news apps.
# In this way, what I expect is to have as antecedent a news app and as consequent a trading app (and/or vice versa).
# This is because in order to trade correctly, one has to read the news to know how the world changes can affect the market.

transactions = [
    ['eToro', 'Yahoo Finance', 'Bloomberg'],
    ['eToro', 'Yahoo Finance', 'Fidelity'],
    ['Yahoo Finance', 'Bloomberg', 'Fidelity'],
    ['eToro', 'Yahoo Finance', 'Wealthfront', 'Bloomberg'],
    ['Investing.com', 'Yahoo Finance', 'Fidelity'],
    ['eToro', 'Yahoo Finance', 'Fidelity', 'Wealthfront'],
    ['eToro', 'Fidelity'],
    ['TradingView', 'Robinhood'],
    ['Robinhood', 'Wealthfront', 'Fidelity'],
    ['Robinhood', 'Wealthsimple', 'Fidelity'],
    ['Wealthsimple', 'Wealthfront', 'Fidelity'],
    ['Seeking Alpha', 'Bloomberg'],
    ['Investing.com', 'Wealthfront', 'Fidelity'],
    ['Wealthfront', 'Robinhood'],
    ['Investing.com', 'Wealthfront', 'TradingView'],
    ['Investing.com', 'TradingView'],
    ['TradingView', 'Robinhood'],
    ['Yahoo Finance', 'Wealthfront'], 
    ['eToro', 'Investing.com'], 
    ['Fidelity', 'Wealthfront']  
]

# Total of transactions: 20


results = list(apriori(transactions, min_support=0.1, min_confidence=0.9, min_lift=1.0))
print_apyori_output(results)

Rules involving itemset ['Wealthsimple', 'Fidelity']
['Wealthsimple'] => ['Fidelity'] (support=0.1000, confidence=1.00, lift=2.00)

Rules involving itemset ['Bloomberg', 'Yahoo Finance', 'eToro']
['Bloomberg', 'eToro'] => ['Yahoo Finance'] (support=0.1000, confidence=1.00, lift=2.86)

Rules involving itemset ['Wealthfront', 'Yahoo Finance', 'eToro']
['Wealthfront', 'eToro'] => ['Yahoo Finance'] (support=0.1000, confidence=1.00, lift=2.86)



<font size="+1" color="red">Replace this cell with a markdown cell containing (1) a printout of the rules you have obtained, and (2) for each of those rules, indicate clearly how the support, confidence, and lift is calculated. Do not merely repeat the formula: indicate how each number is computed based on the transactions you provided, as if you were trying to verify that the numbers are correct.</font>

(1)
RULE 1
Rules involving itemset ['Fidelity', 'Wealthsimple']
['Wealthsimple'] => ['Fidelity'] (support=0.1000, confidence=1.00, lift=2.00)

RULE 2
Rules involving itemset ['Yahoo Finance', 'Bloomberg', 'eToro']
['eToro', 'Bloomberg'] => ['Yahoo Finance'] (support=0.1000, confidence=1.00, lift=2.86)

RULE 3
Rules involving itemset ['Yahoo Finance', 'Wealthfront', 'eToro']
['eToro', 'Wealthfront'] => ['Yahoo Finance'] (support=0.1000, confidence=1.00, lift=2.86)


(2)

RULE 1
The combination ['Wealthsimple', 'Fidelity'] appears in 2 transactions.
Total transactions = 20.
Support = 2/20 = 0.1
Transactions containing ['Wealthsimple'] = 2
Confidence = 2/2 = 1
Support of 'Fidelity' = 10/20 = 0.5 (It appears in 10 transactions from the total of 20)
Lift = 1 / 0.5 = 2

RULE 2
The combination ['eToro', 'Bloomberg', 'Yahoo Finance'] appears in 2 transactions.
Total transactions = 20.
Support = 2/20 = 0.1
Transactions containing ['eToro', 'Bloomberg'] = 2
Confidence = 2/2 = 1
Support of 'Yahoo Finance' = 7/20 = 0.35 (It appears in 7 transactions from the total of 20)
Lift = 1 / 0.35 = 2.86

RULE 3
The combination ['eToro', 'Wealthfront', 'Yahoo Finance'] appears in 2 transactions.
Total transactions = 20.
Support = 2/20 = 0.1
Transactions containing ['eToro', 'Wealthfront'] = 2
Confidence = 2/2 = 1
Support of 'Yahoo Finance' = 7/20 = 0.35
Lift = 1 / 0.35 = 2.86

# 2. Load and prepare the shopping baskets

In [4]:
# LEAVE AS-IS

# File names
INPUT_PRODUCTS = "data/instacart/instacart-products.csv"
INPUT_TRANSACTIONS = "data/instacart/instacart-transactions.csv.gz"

# Read into a dataframe
products = pd.read_csv(INPUT_PRODUCTS, delimiter=",")

# Set product_id as index, and drop column aisle_id
products = products.set_index('product_id').drop(columns=['aisle_id'])

products.head(100)

,product_name,department_id
product_id,,
1,Chocolate Sandwich Cookies,19
2,All-Seasons Salt,13
3,Robust Golden Unsweetened Oolong Tea,7
4,Smart Ones Classic Favorites Mini Rigatoni Wit...,1
5,Green Chile Anytime Sauce,13
...,...,...
96,Sprinklez Confetti Fun Organic Toppings,13
97,Organic Chamomile Lemon Tea,7
98,2% Yellow American Cheese,16


## 2.1. Select by department

In [5]:
# LEAVE AS-IS

DEPT_BAKERY = 3
DEPT_VEGGIES = 4
DEPT_ALCOHOL = 5
DEPT_WORLD = 6
DEPT_DRINKS = 7
DEPT_PETS = 8
DEPT_PHARMACY = 11
DEPT_CLEANING = 17
DEPT_BABIES = 18

<font size="+1" color="red">Replace this cell with your code for *select_from_departments*.</font>

In [6]:
def select_from_departments(products, product_ids, department_ids):
    selected_products = []
    print('TEST CASE:')
    print(product_ids)
    
    # We iterate through all the products to print them as input
    print('\nINPUT PRODUCTS: ')
    for product_id in product_ids:
        print(f'{product_id} {products.loc[product_id].product_name} (dept {products.loc[product_id].department_id})')
    
    # We iterate through all the products and select the department_id
    print('\nSELECTED PRODUCTS: ')
    for product_id in product_ids:
        department_id = products.loc[product_id].department_id
       
        # Then if the id is in the list of department_ids we add it to the selected products list
        if department_id in department_ids:
            print(f'{product_id} {products.loc[product_id].product_name} (dept {department_id})')
            selected_products.append(product_id)
    
    return selected_products   

<font size="+1" color="red">Replace this cell with code to test your function with three different test cases. Each test case is a list of items and a list of 1, 2, or 3 departments.</font>

In [7]:
select_from_departments(products, [22, 26, 45, 54, 83, 57, 71, 111, 112], [DEPT_BAKERY,DEPT_CLEANING])

TEST CASE:
[22, 26, 45, 54, 83, 57, 71, 111, 112]

INPUT PRODUCTS: 
22 Fresh Breath Oral Rinse Mild Mint (dept 11)
26 Fancy Feast Trout Feast Flaked Wet Cat Food (dept 8)
45 European Cucumber (dept 4)
54 24/7 Performance Cat Litter (dept 8)
83 100% Whole Wheat Pita Bread (dept 3)
57 Flat Toothpicks (dept 17)
71 Ultra 7 Inch Polypropylene Traditional Plates (dept 17)
111 Fabric Softener, Geranium Scent (dept 17)
112 Hot Tomatillo Salsa (dept 13)

SELECTED PRODUCTS: 
83 100% Whole Wheat Pita Bread (dept 3)
57 Flat Toothpicks (dept 17)
71 Ultra 7 Inch Polypropylene Traditional Plates (dept 17)
111 Fabric Softener, Geranium Scent (dept 17)


[83, 57, 71, 111]

In [8]:
select_from_departments(products, [11, 59, 873, 879, 882, 97, 35000, 77, 725, 40000, 10,3,4,6], [DEPT_VEGGIES, DEPT_DRINKS, DEPT_BABIES])

TEST CASE:
[11, 59, 873, 879, 882, 97, 35000, 77, 725, 40000, 10, 3, 4, 6]

INPUT PRODUCTS: 
11 Peach Mango Juice (dept 7)
59 Medium Taqueria Style Chipotle Salsa (dept 19)
873 Graduates Lil' Meals Spaghetti Rings In Meat Sauce (dept 18)
879 Baby Dry Diapers Size 4 (dept 18)
882 Fingerling Potato (dept 4)
97 Organic Chamomile Lemon Tea (dept 7)
35000 Apple Frank Organic Beef (dept 21)
77 Coconut Chocolate Chip Energy Bar (dept 19)
725 Ceylon Cinnamon (dept 13)
40000 Select Starts Cranberry Almond Instant Oatmeal (dept 14)
10 Sparkling Orange Juice & Prickly Pear Beverage (dept 7)
3 Robust Golden Unsweetened Oolong Tea (dept 7)
4 Smart Ones Classic Favorites Mini Rigatoni With Vodka Cream Sauce (dept 1)
6 Dry Nose Oil (dept 11)

SELECTED PRODUCTS: 
11 Peach Mango Juice (dept 7)
873 Graduates Lil' Meals Spaghetti Rings In Meat Sauce (dept 18)
879 Baby Dry Diapers Size 4 (dept 18)
882 Fingerling Potato (dept 4)
97 Organic Chamomile Lemon Tea (dept 7)
10 Sparkling Orange Juice & Prickly Pe

[11, 873, 879, 882, 97, 10, 3]

In [9]:
select_from_departments(products, [876, 2345, 2352, 2527, 2534, 3074,36834,36875], [DEPT_ALCOHOL,DEPT_PETS,DEPT_PHARMACY])

TEST CASE:
[876, 2345, 2352, 2527, 2534, 3074, 36834, 36875]

INPUT PRODUCTS: 
876 Vitamin B6 100 mg (dept 11)
2345 Raw Shea Butter Restorative Conditioner (dept 11)
2352 Canine Cuisine with Lamb in Meaty Juices (dept 8)
2527 ProActive Health Adult Hairball Care Dry Cat Food (dept 8)
2534 PM Acetaminophen Caplets (dept 11)
3074 Seafood Sensations Dry Cat Food (dept 8)
36834 Bumpy Bristle Massage Brush (dept 11)
36875 Chablis (dept 5)

SELECTED PRODUCTS: 
876 Vitamin B6 100 mg (dept 11)
2345 Raw Shea Butter Restorative Conditioner (dept 11)
2352 Canine Cuisine with Lamb in Meaty Juices (dept 8)
2527 ProActive Health Adult Hairball Care Dry Cat Food (dept 8)
2534 PM Acetaminophen Caplets (dept 11)
3074 Seafood Sensations Dry Cat Food (dept 8)
36834 Bumpy Bristle Massage Brush (dept 11)
36875 Chablis (dept 5)


[876, 2345, 2352, 2527, 2534, 3074, 36834, 36875]

## 2.2. Read and filter transactions

<font size="+1" color="red">Replace this cell with your code to read transactions, keeping only items in DEPT_PHARMACY. Remember to stop after storing 5000 of the transactions read.</font>

In [10]:
def extract_transaction_from_department(filename, departments):
    # Open a compressed file
    with gzip.open(filename, "rt") as inputfile:

        # Create a CSV reader
        reader = csv.reader(inputfile, delimiter=",")

        transactions = []
        
        # We will use these variables to know the number of transactions read and stored are there at every moment
        read = stored = 0

        # Iterate through the CSV file
        for row in reader:

            # Convert to integers
            items = [int(x) for x in row]
            
            #Iterate through the items and see if any belongs to the corresponding department
            new_items = [item for item in items if products.loc[item].department_id in departments]
            read+=1

            if new_items:
                transactions.append(new_items)
                stored+=1
            
            # Print the stored transactions every 1000 read
            if read % 1000 == 0:
                print(f"Transactions read: {read}, Transactions stored: {stored}")
            
            # Print total transactions and stop when it reaches 5000
            if stored >= 5000:
                print(f'Total transactions read: {read}')
                break
        return transactions
    
transactions = extract_transaction_from_department(INPUT_TRANSACTIONS, [DEPT_PHARMACY])

Transactions read: 1000, Transactions stored: 102
Transactions read: 2000, Transactions stored: 219
Transactions read: 3000, Transactions stored: 313
Transactions read: 4000, Transactions stored: 422
Transactions read: 5000, Transactions stored: 526
Transactions read: 6000, Transactions stored: 619
Transactions read: 7000, Transactions stored: 722
Transactions read: 8000, Transactions stored: 810
Transactions read: 9000, Transactions stored: 920
Transactions read: 10000, Transactions stored: 1046
Transactions read: 11000, Transactions stored: 1152
Transactions read: 12000, Transactions stored: 1245
Transactions read: 13000, Transactions stored: 1352
Transactions read: 14000, Transactions stored: 1459
Transactions read: 15000, Transactions stored: 1559
Transactions read: 16000, Transactions stored: 1663
Transactions read: 17000, Transactions stored: 1753
Transactions read: 18000, Transactions stored: 1854
Transactions read: 19000, Transactions stored: 1943
Transactions read: 20000, Tran

## 2.3. Extract association rules and comment on them

<font size="+1" color="red">Replace this cell with your code to extract association rules from the read transactions.</font>

In [11]:
results1 = list(apriori(transactions, min_support=0.0005, min_confidence=0.749, min_lift=1.0))
print_apyori_output(results1, products, 'product_name')

Rules involving itemset [23425, 5019]
['Nourish & Moisturize Shampoo'] => ['Nourish+ Moisturize Conditioner'] (support=0.0006, confidence=1.00, lift=1250.00)
['Nourish+ Moisturize Conditioner'] => ['Nourish & Moisturize Shampoo'] (support=0.0006, confidence=0.75, lift=1250.00)

Rules involving itemset [11007, 5663]
['Chocolate Energy Supplement'] => ['Chocolate Calming Supplement'] (support=0.0006, confidence=1.00, lift=1250.00)
['Chocolate Calming Supplement'] => ['Chocolate Energy Supplement'] (support=0.0006, confidence=0.75, lift=1250.00)

Rules involving itemset [42563, 10781]
['Coconut Milk Nourishing Shampoo'] => ['Coconut Milk Nourishing Conditioner'] (support=0.0006, confidence=0.75, lift=625.00)

Rules involving itemset [35224, 12246]
['Drink It Up Coconut Milk Natural Hawaiian Shampoo'] => ['Drink It Up Coconut Milk Conditioner'] (support=0.0006, confidence=0.75, lift=340.91)

Rules involving itemset [16122, 44138]
['Plant-Based Vanilla Chai Flavor Nutritional Shake Drink Mi

<font size="+1" color="red">Replace this cell with a brief commentary on what you would recommend to the shopping app considering the extracted association rules.</font>

The extracted association rules highlight interesting relationships among specific products, such as the Nourish & Moisturize Shampoo and Conditioner, which suggests that customers who buy one are likely to purchase the other. This indicates that the app should recommend the conditioner when the shampoo is added to the cart and viceversa. 

Similarly, the association between Chocolate Energy Supplement and Chocolate Calming Supplement implies that customers that buy the energy suplement will also buy the calming one. Therefore, there is an opportunity for the app to recommend the calming supplement when the energy one is added to the cart.

For the other rules we would recommend the same, that the element in the consequent should be recommended to the cart when the element in the antecedent is added to the cart. 

## 2.4. Extract association rules and comment on them (other departments)


<font size="+1" color="red">Replace this cell with code to select a different set of departments (at least two, not DEPT_PHARMACY) and extract transactions again. Avoid replicating code when possible.</font>

In [12]:
transactions_new = extract_transaction_from_department(INPUT_TRANSACTIONS, [DEPT_ALCOHOL,DEPT_BAKERY,DEPT_CLEANING])

results2 = list(apriori(transactions_new, min_support=0.0005, min_confidence=0.749, min_lift=1.0))
print_apyori_output(results2, products, 'product_name')

Transactions read: 1000, Transactions stored: 383
Transactions read: 2000, Transactions stored: 777
Transactions read: 3000, Transactions stored: 1201
Transactions read: 4000, Transactions stored: 1584
Transactions read: 5000, Transactions stored: 1990
Transactions read: 6000, Transactions stored: 2377
Transactions read: 7000, Transactions stored: 2779
Transactions read: 8000, Transactions stored: 3166
Transactions read: 9000, Transactions stored: 3578
Transactions read: 10000, Transactions stored: 3974
Transactions read: 11000, Transactions stored: 4328
Transactions read: 12000, Transactions stored: 4710
Total transactions read: 12736
Rules involving itemset [46184, 24533]
['Pre-Moistened Multi-Surface Everyday Wipes'] => ['Original Pre-Moistened Wipes'] (support=0.0006, confidence=0.75, lift=1250.00)
['Original Pre-Moistened Wipes'] => ['Pre-Moistened Multi-Surface Everyday Wipes'] (support=0.0006, confidence=1.00, lift=1250.00)



<font size="+1" color="red">Replace this cell with your commentary on the obtained rules.</font>

For the rules involving Pre-Moistened Multi-Surface Everyday Wipes and Original Pre-Moistened Wipes, we observe the same pattern as before. Customers purchasing the pre-moistened wipe are going to buy the other as well, since they have a confidence of 1.0. Therefore, it would be beneficial for the app to recommend the Original Pre-Moistened Wipes when the Pre-Moistened Multi-Surface Everyday Wipes are added to the cart, and vice versa. 

<font size="+2" color="#003300">I hereby declare that, except for the code provided by the course instructors, all of my code, report, and figures were produced by myself.</font>